# Module: Agent Hooks for Execution Control in LangChain 1.0

**Building on Previous HR Modules:**
- Module 3: Created HR tools with @tool decorator
- Module 4: Added short-term memory and persistence
- **This Module: Control agent execution with hook functions**

**What you'll learn:**
- 🎯 pre_model_hook and post_model_hook patterns
- 🔒 Access control and authorization
- ⏱️ Rate limiting and throttling
- 📊 Logging and monitoring
- 🛡️ Input validation and sanitization
- 💰 Token usage tracking and cost management
- 🚨 Error handling and recovery
- ✅ Approval workflows (human-in-the-loop)
- 🔗 Chaining multiple hooks for production systems

**Real HR Use Cases:**
- Verify employee authorization before sensitive operations
- Log all HR queries for compliance
- Rate limit requests to prevent abuse
- Validate and sanitize employee inputs
- Monitor token usage and costs
- Implement approval workflows for critical actions

**Time:** 2-3 hours

## Setup: Install Dependencies

In [ ]:
# Install LangChain 1.0 alpha packages
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configure API Keys & Imports

In [ ]:
# Configure API key
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Common imports
from langchain.agents import create_agent, AgentState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from typing import Annotated, TypedDict, Literal
from datetime import datetime
import json

print("✅ Setup complete!")
print("\n📌 IMPORTANT NOTE:")
print("The LangChain documentation shows AgentMiddleware classes, but in LangChain 1.0,")
print("we use simple hook FUNCTIONS instead:")
print("  • pre_model_hook: runs before LLM")
print("  • post_model_hook: runs after LLM")
print("\nThis notebook uses the correct LangChain 1.0 patterns!")

---
# Part 1: Understanding Hooks 🎯

## What are Hooks?

Hooks are functions that intercept the agent execution flow:

```
User Input → [pre_model_hook] → LLM → [post_model_hook] → Tools → Response
              ↑                         ↑
              Pre-processing            Post-processing
```

## Two Types of Hooks in LangChain 1.0

### 1. **pre_model_hook** 
- Runs BEFORE the LLM is called
- Can update state permanently
- Can jump to different nodes (using `jump_to: "__end__"`)
- Use for: authorization, logging, validation, rate limiting
- Signature: `def hook(state: AgentState) -> dict`

### 2. **post_model_hook**
- Runs AFTER the LLM responds
- Can update state permanently
- Can jump to different nodes
- Use for: response validation, logging, metrics, content filtering
- Signature: `def hook(state: AgentState) -> dict`

## Hook Return Values

Hooks return a dictionary with state updates:

```python
# Normal flow - return empty dict or state updates
return {"key": "value"}

# Early exit - jump to end
return {
    "messages": [("assistant", "Access denied")],
    "jump_to": "__end__"  # Skip LLM/tools and end immediately
}
```

## Setup: HR Tools for Examples

In [ ]:
# HR Employee Database (mock)
EMPLOYEES = {
    "101": {"name": "Priya Sharma", "department": "Engineering", "role": "Senior Developer", "leave_balance": 12},
    "102": {"name": "Rahul Verma", "department": "Engineering", "role": "Manager", "leave_balance": 8},
    "103": {"name": "Anjali Patel", "department": "HR", "role": "Director", "leave_balance": 15},
    "104": {"name": "Arjun Reddy", "department": "Sales", "role": "Team Lead", "leave_balance": 10},
    "105": {"name": "Sneha Gupta", "department": "Marketing", "role": "Specialist", "leave_balance": 5}
}

# Define HR tools
@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID to look up"]) -> str:
    """Get employee information by ID."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} - {emp['department']} - {emp['role']}"
    return f"Employee {employee_id} not found"

@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Check leave balance for an employee."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"{emp['name']} has {emp['leave_balance']} days of leave remaining"
    return f"Employee {employee_id} not found"

@tool
def update_salary(employee_id: Annotated[str, "Employee ID"], 
                  new_salary: Annotated[float, "New salary amount"]) -> str:
    """Update employee salary. SENSITIVE OPERATION - requires authorization."""
    if employee_id in EMPLOYEES:
        return f"Salary updated for {EMPLOYEES[employee_id]['name']} to ₹{new_salary:,.2f}"
    return f"Employee {employee_id} not found"

print("✅ HR tools defined!")
print(f"Available tools: {get_employee_info.name}, {check_leave_balance.name}, {update_salary.name}")

---
# Part 2: pre_model_hook - Pre-processing

**Use pre_model_hook for:**
- Authorization checks
- Input validation
- Logging requests
- Rate limiting
- Early exit conditions

**Key Pattern:**
```python
def my_pre_hook(state: AgentState) -> dict:
    # Do validation/checks
    if should_block:
        return {
            "messages": [("assistant", "Error message")],
            "jump_to": "__end__"  # Skip LLM
        }
    return {}  # Continue normally
```

## Lab 2.1: Basic Logging Middleware

In [ ]:
from langchain.agents import AgentMiddleware

class LoggingMiddleware(AgentMiddleware):
    """Log all requests before they reach the LLM."""
    
    def before_model(self, state: AgentState) -> dict:
        """Log the incoming message before LLM processing."""
        messages = state.get("messages", [])
        if messages:
            last_message = messages[-1]
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(f"\n📝 [LOGGING MIDDLEWARE - {timestamp}]")
            print(f"   User: {last_message.content}")
        
        # Return empty dict - no state modification
        return {}

# Create agent with logging middleware
logging_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info],
    middleware=[LoggingMiddleware()],
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Logging Middleware")
print("=" * 70)

result = logging_agent.invoke({
    "messages": [{"role": "user", "content": "Who is employee 101?"}]
})

print(f"\nAgent Response: {result['messages'][-1].content}")
print("\n✅ All requests are now logged!")

## Lab 2.2: Authorization Middleware

In [ ]:
# Custom state with user context
class AuthorizedAgentState(AgentState):
    current_user_id: str = ""
    current_user_role: str = ""
    authorized: bool = False

class AuthorizationMiddleware(AgentMiddleware):
    """Check if user is authorized for sensitive operations."""
    
    def __init__(self, sensitive_tools: list[str]):
        self.sensitive_tools = sensitive_tools
    
    def before_model(self, state: AuthorizedAgentState) -> dict:
        """Verify authorization before processing."""
        user_role = state.get("current_user_role", "")
        user_id = state.get("current_user_id", "unknown")
        
        print(f"\n🔐 [AUTHORIZATION CHECK]")
        print(f"   User ID: {user_id}")
        print(f"   Role: {user_role}")
        
        # Check if message mentions sensitive operations
        messages = state.get("messages", [])
        if messages:
            content = messages[-1].content.lower()
            needs_auth = any(tool in content for tool in ["salary", "update", "change"])
            
            if needs_auth:
                # Only HR Directors can update salaries
                if user_role == "HR Director":
                    print(f"   ✅ Authorized - {user_role} can perform sensitive operations")
                    return {"authorized": True}
                else:
                    print(f"   ❌ UNAUTHORIZED - {user_role} cannot perform this operation")
                    # Jump directly to end, skip LLM
                    return {
                        "authorized": False,
                        "messages": [("assistant", f"❌ Access Denied: Only HR Directors can perform salary updates. Your role: {user_role}")],
                        "jump_to": "__end__"  # Skip LLM and tools!
                    }
        
        return {"authorized": True}

# Create agent with authorization
auth_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, update_salary],
    middleware=[AuthorizationMiddleware(sensitive_tools=["update_salary"])],
    state_schema=AuthorizedAgentState,
    prompt="You are an HR assistant. Help with employee queries."
)

print("=" * 70)
print("Test 1: Unauthorized User (Regular Employee)")
print("=" * 70)

result = auth_agent.invoke({
    "messages": [{"role": "user", "content": "Update salary for employee 101 to 150000"}],
    "current_user_id": "104",
    "current_user_role": "Team Lead"
})
print(f"Response: {result['messages'][-1].content}")

print("\n" + "=" * 70)
print("Test 2: Authorized User (HR Director)")
print("=" * 70)

result = auth_agent.invoke({
    "messages": [{"role": "user", "content": "Update salary for employee 101 to 150000"}],
    "current_user_id": "103",
    "current_user_role": "HR Director"
})
print(f"Response: {result['messages'][-1].content}")

print("\n✅ Authorization middleware protects sensitive operations!")

## Lab 2.3: Rate Limiting Middleware

In [ ]:
from collections import defaultdict
from datetime import datetime, timedelta

class RateLimitMiddleware(AgentMiddleware):
    """Limit number of requests per user per time window."""
    
    def __init__(self, max_requests: int = 5, window_seconds: int = 60):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.request_history = defaultdict(list)  # user_id -> [timestamps]
    
    def before_model(self, state: AgentState) -> dict:
        """Check rate limit before processing."""
        user_id = state.get("current_user_id", "anonymous")
        now = datetime.now()
        
        # Clean old requests outside the time window
        cutoff = now - timedelta(seconds=self.window_seconds)
        self.request_history[user_id] = [
            ts for ts in self.request_history[user_id] if ts > cutoff
        ]
        
        # Check if limit exceeded
        current_count = len(self.request_history[user_id])
        
        print(f"\n⏱️  [RATE LIMIT CHECK]")
        print(f"   User: {user_id}")
        print(f"   Requests in last {self.window_seconds}s: {current_count}/{self.max_requests}")
        
        if current_count >= self.max_requests:
            print(f"   ❌ RATE LIMIT EXCEEDED")
            return {
                "messages": [("assistant", f"⏱️ Rate limit exceeded. You have made {current_count} requests in the last {self.window_seconds} seconds. Maximum allowed: {self.max_requests}. Please try again later.")],
                "jump_to": "__end__"
            }
        
        # Add current request
        self.request_history[user_id].append(now)
        print(f"   ✅ Request allowed ({current_count + 1}/{self.max_requests})")
        return {}

# Create agent with rate limiting
rate_limited_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info],
    middleware=[RateLimitMiddleware(max_requests=3, window_seconds=60)],
    state_schema=AuthorizedAgentState,
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Rate Limiting (3 requests per 60 seconds)")
print("=" * 70)

# Make 4 requests quickly
for i in range(4):
    print(f"\n--- Request {i+1} ---")
    result = rate_limited_agent.invoke({
        "messages": [{"role": "user", "content": f"Who is employee 10{i+1}?"}],
        "current_user_id": "104"
    })
    print(f"Response: {result['messages'][-1].content[:100]}...")

print("\n✅ Rate limiting prevents abuse!")

## Lab 2.4: Input Validation Middleware

In [ ]:
import re

class InputValidationMiddleware(AgentMiddleware):
    """Validate and sanitize user inputs."""
    
    def before_model(self, state: AgentState) -> dict:
        """Validate input before processing."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        last_message = messages[-1]
        content = last_message.content
        
        print(f"\n🛡️  [INPUT VALIDATION]")
        
        # Check for empty input
        if not content or not content.strip():
            print("   ❌ Empty input detected")
            return {
                "messages": [("assistant", "Please provide a valid question or request.")],
                "jump_to": "__end__"
            }
        
        # Check for SQL injection attempts (basic)
        sql_patterns = [r"DROP\s+TABLE", r"DELETE\s+FROM", r"INSERT\s+INTO", r"--", r";"]
        for pattern in sql_patterns:
            if re.search(pattern, content, re.IGNORECASE):
                print(f"   ❌ Potential SQL injection detected: {pattern}")
                return {
                    "messages": [("assistant", "⚠️ Invalid input detected. Please rephrase your request.")],
                    "jump_to": "__end__"
                }
        
        # Check for excessive length
        if len(content) > 1000:
            print(f"   ❌ Input too long: {len(content)} characters")
            return {
                "messages": [("assistant", "Your message is too long. Please keep it under 1000 characters.")],
                "jump_to": "__end__"
            }
        
        # Sanitize: remove special characters (keep basic punctuation)
        sanitized = re.sub(r'[^a-zA-Z0-9\s.,?!-]', '', content)
        if sanitized != content:
            print(f"   🧹 Input sanitized (removed special characters)")
            # Update the message with sanitized content
            messages[-1].content = sanitized
            return {"messages": messages}
        
        print("   ✅ Input valid")
        return {}

# Create agent with input validation
validated_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info],
    middleware=[InputValidationMiddleware()],
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Input Validation")
print("=" * 70)

# Test 1: Valid input
print("\nTest 1: Valid input")
result = validated_agent.invoke({
    "messages": [{"role": "user", "content": "Who is employee 101?"}]
})
print(f"Response: {result['messages'][-1].content[:100]}")

# Test 2: SQL injection attempt
print("\nTest 2: SQL injection attempt")
result = validated_agent.invoke({
    "messages": [{"role": "user", "content": "SELECT * FROM employees; DROP TABLE users;"}]
})
print(f"Response: {result['messages'][-1].content}")

# Test 3: Empty input
print("\nTest 3: Empty input")
result = validated_agent.invoke({
    "messages": [{"role": "user", "content": "   "}]
})
print(f"Response: {result['messages'][-1].content}")

print("\n✅ Input validation protects against malicious inputs!")

---
# Part 3: Dynamic Context with pre_model_hook

**Pattern:** Inject context dynamically before LLM processing

While LangChain 1.0's `create_agent` doesn't have `modify_model_request`, you can achieve similar functionality by:
1. Using `pre_model_hook` to add context to state
2. Using function-based prompts that read from state
3. Adding system messages dynamically

**Use for:**
- User-aware responses
- Dynamic context injection
- Personalization
- Session-specific instructions

## Lab 3.1: Dynamic Context Injection

In [ ]:
from langchain.agents import AgentMiddleware, ModelRequest
from langchain_core.messages import SystemMessage

class ContextInjectionMiddleware(AgentMiddleware):
    """Inject user context into the system prompt dynamically."""
    
    def modify_model_request(self, state: AgentState, request: ModelRequest) -> ModelRequest:
        """Add employee context to system prompt."""
        user_id = state.get("current_user_id", "unknown")
        user_role = state.get("current_user_role", "unknown")
        
        # Get employee details
        emp_info = ""
        if user_id in EMPLOYEES:
            emp = EMPLOYEES[user_id]
            emp_info = f"Current User: {emp['name']} (ID: {user_id})\nRole: {emp['role']}\nDepartment: {emp['department']}"
        
        # Inject context into system prompt
        original_prompt = request.system_prompt or ""
        enhanced_prompt = f"""{original_prompt}

CURRENT USER CONTEXT:
{emp_info}

Always be aware of who you're talking to and personalize responses accordingly.
"""
        
        print(f"\n💉 [CONTEXT INJECTION]")
        print(f"   Injected context for: {user_id} ({user_role})")
        
        request.system_prompt = enhanced_prompt
        return request

# Create agent with context injection
context_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[check_leave_balance],
    middleware=[ContextInjectionMiddleware()],
    state_schema=AuthorizedAgentState,
    prompt="You are a helpful HR assistant."
)

print("=" * 70)
print("Testing Context Injection")
print("=" * 70)

result = context_agent.invoke({
    "messages": [{"role": "user", "content": "How many leave days do I have?"}],
    "current_user_id": "101",
    "current_user_role": "Senior Developer"
})

print(f"\nResponse: {result['messages'][-1].content}")
print("\n✅ Agent knows who it's talking to without explicit mention!")

## Lab 3.2: Dynamic Model Switching

In [ ]:
# Model switching based on query complexity
# Note: In LangChain 1.0, model is set at agent creation
# Dynamic model switching requires custom graph implementation

def detect_query_complexity(content: str) -> str:
    """Detect if query is simple or complex."""
    complex_keywords = ["analyze", "compare", "evaluate", "recommend", "strategy", "plan"]
    return "complex" if any(keyword in content.lower() for keyword in complex_keywords) else "simple"

def model_selection_hook(state: AgentState) -> dict:
    """Log which model would be ideal for this query."""
    messages = state.get("messages", [])
    if not messages:
        return {}
    
    content = messages[-1].content
    complexity = detect_query_complexity(content)
    
    print(f"\n🔄 [MODEL SELECTION ANALYSIS]")
    print(f"   Query complexity: {complexity}")
    if complexity == "complex":
        print(f"   Recommended: GPT-4 for complex analysis")
    else:
        print(f"   Recommended: GPT-3.5 for simple queries (cost optimization)")
    
    return {}

# Note: This demonstrates the concept. In production, you would:
# 1. Use LangGraph to create custom routing logic
# 2. Create separate agents for different models
# 3. Route based on complexity analysis

print("✅ Model selection logic defined!")
print("💡 In production, use LangGraph conditional edges for true model switching")

---
# Part 4: post_model_hook - Post-processing

**Use post_model_hook for:**
- Response validation
- Content filtering
- Logging responses
- Token usage tracking
- Error recovery
- Approval workflows

**Key Pattern:**
```python
def my_post_hook(state: AgentState) -> dict:
    # Process the response
    messages = state.get("messages", [])
    
    # Log, validate, or modify
    # ...
    
    return {}  # Or state updates
```

## Lab 4.1: Response Logging and Audit Trail

In [ ]:
class AuditTrailMiddleware(AgentMiddleware):
    """Log all interactions for compliance and auditing."""
    
    def __init__(self):
        self.audit_log = []
    
    def before_model(self, state: AgentState) -> dict:
        """Log request."""
        messages = state.get("messages", [])
        if messages:
            self.request_timestamp = datetime.now()
            self.request_content = messages[-1].content
        return {}
    
    def after_model(self, state: AgentState) -> dict:
        """Log response and save audit entry."""
        messages = state.get("messages", [])
        if len(messages) >= 2:
            response_content = messages[-1].content
            
            audit_entry = {
                "timestamp": self.request_timestamp.isoformat(),
                "user_id": state.get("current_user_id", "unknown"),
                "request": self.request_content,
                "response": response_content[:100],  # Truncate for readability
                "tools_used": [msg.name for msg in messages if hasattr(msg, 'name')]
            }
            
            self.audit_log.append(audit_entry)
            
            print(f"\n📋 [AUDIT LOG ENTRY CREATED]")
            print(f"   Timestamp: {audit_entry['timestamp']}")
            print(f"   User: {audit_entry['user_id']}")
            print(f"   Request: {audit_entry['request'][:50]}...")
        
        return {}
    
    def get_audit_log(self):
        """Retrieve audit log."""
        return self.audit_log

# Create agent with audit trail
audit_middleware = AuditTrailMiddleware()
audit_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance],
    middleware=[audit_middleware],
    state_schema=AuthorizedAgentState,
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Audit Trail")
print("=" * 70)

# Make several requests
requests = [
    ("Who is employee 101?", "101"),
    ("Check leave balance for employee 102", "102"),
    ("Tell me about employee 103", "103")
]

for query, user_id in requests:
    result = audit_agent.invoke({
        "messages": [{"role": "user", "content": query}],
        "current_user_id": user_id
    })

# Display audit log
print("\n" + "=" * 70)
print("COMPLETE AUDIT LOG")
print("=" * 70)
for i, entry in enumerate(get_audit_log(), 1):
    print(f"\nEntry {i}:")
    print(json.dumps(entry, indent=2))

print("\n✅ All interactions logged for compliance!")

## Lab 4.2: Token Usage Tracking and Cost Management

In [ ]:
class TokenTrackingMiddleware(AgentMiddleware):
    """Track token usage and estimated costs."""
    
    def __init__(self, cost_per_1k_tokens: float = 0.002):
        self.total_tokens = 0
        self.total_cost = 0.0
        self.cost_per_1k = cost_per_1k_tokens
        self.usage_history = []
    
    def after_model(self, state: AgentState) -> dict:
        """Estimate token usage after LLM call."""
        messages = state.get("messages", [])
        
        # Rough estimation (in production, use actual token counts from API)
        estimated_tokens = sum(len(msg.content.split()) * 1.3 for msg in messages)
        estimated_tokens = int(estimated_tokens)
        
        cost = (estimated_tokens / 1000) * self.cost_per_1k
        
        self.total_tokens += estimated_tokens
        self.total_cost += cost
        
        usage_entry = {
            "timestamp": datetime.now().isoformat(),
            "user_id": state.get("current_user_id", "unknown"),
            "tokens": estimated_tokens,
            "cost": f"${cost:.4f}"
        }
        self.usage_history.append(usage_entry)
        
        print(f"\n💰 [TOKEN USAGE]")
        print(f"   This call: ~{estimated_tokens} tokens (${cost:.4f})")
        print(f"   Total: ~{self.total_tokens} tokens (${self.total_cost:.4f})")
        
        return {}
    
    def get_usage_report(self):
        """Generate usage report."""
        return {
            "total_tokens": self.total_tokens,
            "total_cost": f"${self.total_cost:.4f}",
            "history": self.usage_history
        }

# Create agent with token tracking
token_middleware = TokenTrackingMiddleware()
cost_tracking_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info],
    middleware=[token_middleware],
    state_schema=AuthorizedAgentState,
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Token Usage Tracking")
print("=" * 70)

# Make several requests
for i in range(3):
    result = cost_tracking_agent.invoke({
        "messages": [{"role": "user", "content": f"Tell me about employee 10{i+1}"}],
        "current_user_id": f"10{i+1}"
    })

# Display usage report
print("\n" + "=" * 70)
print("USAGE REPORT")
print("=" * 70)
report = get_usage_report()
print(json.dumps(report, indent=2))

print("\n✅ Token usage and costs tracked!")

## Lab 4.3: Content Filtering and Safety

In [ ]:
class ContentFilterMiddleware(AgentMiddleware):
    """Filter responses for sensitive information."""
    
    def __init__(self):
        # Patterns to redact (for demo purposes)
        self.sensitive_patterns = {
            r'\b\d{10}\b': '[PHONE_REDACTED]',  # Phone numbers
            r'\b[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}\b': '[EMAIL_REDACTED]',  # Emails
            r'\b\d{3}-\d{2}-\d{4}\b': '[SSN_REDACTED]',  # SSN format
        }
    
    def after_model(self, state: AgentState) -> dict:
        """Filter response for sensitive data."""
        messages = state.get("messages", [])
        if not messages:
            return {}
        
        last_message = messages[-1]
        original_content = last_message.content
        filtered_content = original_content
        
        # Apply filters
        redacted = False
        for pattern, replacement in self.sensitive_patterns.items():
            matches = re.findall(pattern, filtered_content, re.IGNORECASE)
            if matches:
                filtered_content = re.sub(pattern, replacement, filtered_content, flags=re.IGNORECASE)
                redacted = True
        
        if redacted:
            print(f"\n🛡️  [CONTENT FILTER]")
            print(f"   ⚠️  Sensitive information detected and redacted")
            last_message.content = filtered_content
            return {"messages": messages}
        
        return {}

# Create agent with content filtering
filtered_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[],
    middleware=[ContentFilterMiddleware()],
    prompt="You are an HR assistant. When providing examples, use realistic sample data."
)

print("=" * 70)
print("Testing Content Filtering")
print("=" * 70)

result = filtered_agent.invoke({
    "messages": [{"role": "user", "content": "Can you give me an example employee contact? Use format: Name, email, phone"}]
})

print(f"\nResponse: {result['messages'][-1].content}")
print("\n✅ Sensitive information automatically redacted!")

---
# Part 5: Chaining Multiple Hooks

**Production Pattern:** Combine multiple operations into single hook functions.

Since `create_agent` accepts one `pre_model_hook` and one `post_model_hook`, you need to chain operations:

```python
def combined_pre_hook(state: AgentState) -> dict:
    # Run multiple checks in sequence
    result = logging_hook(state)
    if result.get("jump_to"): return result
    
    result = auth_hook(state)
    if result.get("jump_to"): return result
    
    # ... more hooks
    return {}
```

## Lab 5.1: Production HR Agent with All Middleware

In [ ]:
# Chaining multiple hooks - combine them into single functions
def combined_pre_hook(state: AgentState) -> dict:
    """Chain multiple pre-model hooks."""
    # 1. Logging
    result = logging_hook(state)
    if result.get("jump_to"): return result
    
    # 2. Input Validation
    result = input_validation_hook(state)
    if result.get("jump_to"): return result
    
    # 3. Authorization
    result = authorization_hook(state)
    if result.get("jump_to"): return result
    
    # 4. Rate Limiting
    result = rate_limit_hook(state, max_requests=10, window_seconds=60)
    if result.get("jump_to"): return result
    
    # 5. Context Injection
    result = context_injection_hook(state)
    if result.get("jump_to"): return result
    
    # 6. Audit (pre)
    result = audit_pre_hook(state)
    
    return result

def combined_post_hook(state: AgentState) -> dict:
    """Chain multiple post-model hooks."""
    # 1. Audit (post)
    result = audit_post_hook(state)
    
    # 2. Token Tracking
    result = token_tracking_hook(state)
    
    # 3. Content Filtering
    result = content_filter_hook(state)
    if result.get("messages"):
        return result
    
    return {}

# Create production agent with all hooks
production_hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, check_leave_balance, update_salary],
    pre_model_hook=combined_pre_hook,
    post_model_hook=combined_post_hook,
    state_schema=AuthorizedAgentState,
    checkpointer=InMemorySaver(),
    prompt="""You are a professional HR assistant for a large organization.
    
    Your responsibilities:
    - Help employees with HR queries
    - Check leave balances
    - Provide employee information
    - Handle sensitive operations with care
    
    Always be professional, helpful, and security-conscious."""
)

print("✅ Production HR agent created with multi-layered hook chain!")
print("\nHook layers:")
print("  Pre-model: 📝 Logging → 🛡️ Validation → 🔐 Auth → ⏱️ Rate Limit → 💉 Context → 📋 Audit")
print("  Post-model: 📋 Audit → 💰 Token Tracking → 🛡️ Content Filter")

## Lab 5.2: Test Production Agent

In [ ]:
config = {"configurable": {"thread_id": "production_test_1"}}

print("=" * 70)
print("PRODUCTION HR AGENT TEST")
print("=" * 70)

# Test 1: Normal query (authorized user)
print("\n" + "=" * 70)
print("Test 1: Employee checking own leave balance")
print("=" * 70)
result = production_hr_agent.invoke({
    "messages": [{"role": "user", "content": "How many leave days do I have remaining?"}],
    "current_user_id": "101",
    "current_user_role": "Senior Developer"
}, config)
print(f"\n👤 User: Priya Sharma (101)")
print(f"🤖 Response: {result['messages'][-1].content}")

# Test 2: Unauthorized operation
print("\n" + "=" * 70)
print("Test 2: Unauthorized salary update attempt")
print("=" * 70)
result = production_hr_agent.invoke({
    "messages": [{"role": "user", "content": "Update my salary to 200000"}],
    "current_user_id": "104",
    "current_user_role": "Team Lead"
}, config)
print(f"\n👤 User: Arjun Reddy (104)")
print(f"🤖 Response: {result['messages'][-1].content}")

# Test 3: Authorized operation
print("\n" + "=" * 70)
print("Test 3: Authorized salary update (HR Director)")
print("=" * 70)
result = production_hr_agent.invoke({
    "messages": [{"role": "user", "content": "Update salary for employee 101 to 180000"}],
    "current_user_id": "103",
    "current_user_role": "HR Director"
}, config)
print(f"\n👤 User: Anjali Patel (103)")
print(f"🤖 Response: {result['messages'][-1].content}")

# Display final reports
print("\n" + "=" * 70)
print("AUDIT & USAGE REPORTS")
print("=" * 70)

print("\n📋 Audit Log:")
for i, entry in enumerate(get_audit_log()[-3:], 1):  # Show last 3 entries
    print(f"\n  Entry {i}:")
    print(f"    User: {entry['user_id']}")
    print(f"    Request: {entry['request'][:60]}...")
    print(f"    Time: {entry['timestamp']}")

print("\n💰 Token Usage:")
usage = get_usage_report()
print(f"  Total Tokens: {usage['total_tokens']}")
print(f"  Total Cost: {usage['total_cost']}")

print("\n✅ Production agent with complete hook protection!")

---
# Part 6: Human-in-the-Loop with post_model_hook

**Use Case:** Require human approval for critical operations.

**Pattern:** Intercept tool calls in `post_model_hook` before they execute.

## Lab 6.1: Approval Workflow Middleware

In [ ]:
class ApprovalWorkflowMiddleware(AgentMiddleware):
    """Require approval for sensitive operations."""
    
    def __init__(self, approval_required_tools: list[str]):
        self.approval_required = approval_required_tools
        self.pending_approvals = {}
    
    def after_model(self, state: AgentState) -> dict:
        """Check if tools require approval."""
        messages = state.get("messages", [])
        
        # Check if any tool calls are pending
        for msg in messages:
            if hasattr(msg, 'tool_calls'):
                for tool_call in msg.tool_calls:
                    tool_name = tool_call.get('name', '')
                    
                    if tool_name in self.approval_required:
                        approval_id = f"approval_{len(self.pending_approvals)}"
                        
                        self.pending_approvals[approval_id] = {
                            "tool": tool_name,
                            "args": tool_call.get('args', {}),
                            "status": "pending",
                            "timestamp": datetime.now().isoformat()
                        }
                        
                        print(f"\n✋ [APPROVAL REQUIRED]")
                        print(f"   Tool: {tool_name}")
                        print(f"   Args: {tool_call.get('args', {})}")
                        print(f"   Approval ID: {approval_id}")
                        
                        # In production, this would pause execution
                        # and wait for human approval
                        return {
                            "messages": [("assistant", f"⏸️  Operation requires approval. Approval ID: {approval_id}. Please have a manager review and approve this action.")],
                            "jump_to": "__end__"
                        }
        
        return {}
    
    def approve(self, approval_id: str, approved: bool):
        """Approve or reject a pending operation."""
        if approval_id in self.pending_approvals:
            self.pending_approvals[approval_id]["status"] = "approved" if approved else "rejected"
            return True
        return False
    
    def get_pending_approvals(self):
        return {k: v for k, v in self.pending_approvals.items() if v["status"] == "pending"}

# Create agent with approval workflow
approval_mw = ApprovalWorkflowMiddleware(approval_required_tools=["update_salary"])
approval_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_employee_info, update_salary],
    middleware=[approval_mw],
    state_schema=AuthorizedAgentState,
    prompt="You are an HR assistant."
)

print("=" * 70)
print("Testing Approval Workflow")
print("=" * 70)

result = approval_agent.invoke({
    "messages": [{"role": "user", "content": "Update salary for employee 101 to 200000"}],
    "current_user_id": "103",
    "current_user_role": "HR Director"
})

print(f"\nAgent Response: {result['messages'][-1].content}")

print("\n📋 Pending Approvals:")
for approval_id, details in get_pending_approvals().items():
    print(f"\n  {approval_id}:")
    print(f"    Tool: {details['tool']}")
    print(f"    Args: {details['args']}")
    print(f"    Status: {details['status']}")

print("\n✅ Sensitive operations require human approval!")

---
# Summary & Best Practices

## Hook Execution in LangChain 1.0

```
User Input → pre_model_hook → LLM → post_model_hook → Tools → Response
             ↑                        ↑
             Pre-processing           Post-processing
```

## When to Use Each Hook

| Hook | Use For | Can Modify State | Can Jump |
|------|---------|-----------------|----------|
| **pre_model_hook** | Authorization, validation, logging, rate limiting | ✅ Yes | ✅ Yes |
| **post_model_hook** | Response filtering, audit, metrics, content safety | ✅ Yes | ✅ Yes |

## Production Hook Chain (Recommended Order)

### Pre-Model Hooks:
1. **Logging** - Log all incoming requests
2. **Input Validation** - Sanitize and validate inputs
3. **Rate Limiting** - Prevent abuse
4. **Authorization** - Check permissions
5. **Context Injection** - Add user context
6. **Audit (Pre)** - Record request

### Post-Model Hooks:
1. **Audit (Post)** - Record response
2. **Token Tracking** - Monitor usage and costs
3. **Content Filtering** - Redact sensitive data
4. **Approval Workflow** - Human-in-the-loop for critical ops

## Key Takeaways

✅ **Hooks provide powerful execution control**  
✅ **Chain multiple operations in single hook functions**  
✅ **Use pre_model_hook for pre-processing and authorization**  
✅ **Use post_model_hook for post-processing and auditing**  
✅ **Always log and audit in production**  
✅ **Implement rate limiting to prevent abuse**  
✅ **Track token usage for cost management**  
✅ **Handle `jump_to` for early exits**  

## Important Notes for LangChain 1.0

- `create_agent` accepts **one** `pre_model_hook` and **one** `post_model_hook`
- To chain multiple operations, combine them into single hook functions
- Check for `jump_to` in results to enable early exits
- Hooks receive `AgentState` and return `dict` with state updates

## Next Steps

- Implement custom hooks for your use case
- Add persistent storage for audit logs
- Integrate with external authentication systems
- Build approval workflows with notification systems
- Deploy with production monitoring and alerting

---
# Exercises

## Exercise 1: Department-Based Authorization

Create middleware that:
- Only allows HR department to view salary information
- Only allows managers to approve leave requests
- Logs unauthorized access attempts

**Hint:** Use `before_model` hook with user role checking.

In [ ]:
# Your code here
def department_auth_hook(state: AgentState) -> dict:
    """TODO: Implement department-based authorization."""
    # TODO: Check user department and role
    # TODO: Verify permissions for specific operations
    # TODO: Log unauthorized attempts
    pass

# TODO: Create tools for salary info and leave approval
# TODO: Test with different user roles

## Exercise 2: Smart Caching Middleware

Create middleware that:
- Caches responses for identical queries
- Invalidates cache after 5 minutes
- Logs cache hits/misses
- Calculates cache hit rate

**Hint:** Use `before_model` to check cache, `after_model` to store responses.

In [ ]:
# Your code here
from datetime import datetime, timedelta

cache_data = {
    'cache': {},  # query -> (response, timestamp)
    'hits': 0,
    'misses': 0
}

def caching_pre_hook(state: AgentState, ttl_seconds: int = 300) -> dict:
    """TODO: Check cache before LLM call."""
    # TODO: Hash the query
    # TODO: Check if cached and not expired
    # TODO: If cache hit, return cached response and jump to end
    # TODO: Track cache hits
    pass

def caching_post_hook(state: AgentState, ttl_seconds: int = 300) -> dict:
    """TODO: Store response in cache."""
    # TODO: Store response with timestamp
    # TODO: Track cache misses
    pass

## Exercise 3: Multi-Language Support

Create middleware that:
- Detects user's preferred language from state
- Modifies system prompt to include language instruction
- Translates common HR terms

**Hint:** Use `modify_model_request` hook.

**Languages to support:** English, Hindi, Tamil

In [ ]:
# Your code here
def multi_language_hook(state: AgentState) -> dict:
    """TODO: Add multi-language support."""
    # TODO: Get user's preferred language from state
    # TODO: Add language-specific instructions to messages
    # TODO: Include translations for common HR terms
    pass

# Note: In LangChain 1.0, modify the messages in state
# or add a system message with language instructions

## Exercise 4: Compliance Reporting

Create middleware that generates compliance reports:
- Track all data access by employee
- Log sensitive operations (salary updates, leave approvals)
- Generate daily summary reports
- Alert on suspicious patterns (e.g., excessive queries)

**Hint:** Combine `before_model` and `after_model` hooks.

In [ ]:
# Your code here
compliance_data = {
    'access_log': [],
    'sensitive_ops': []
}

def compliance_pre_hook(state: AgentState) -> dict:
    """TODO: Track data access attempts."""
    # TODO: Log who accessed what data
    # TODO: Record timestamp and user info
    # TODO: Detect suspicious patterns (e.g., excessive queries)
    pass

def compliance_post_hook(state: AgentState) -> dict:
    """TODO: Log sensitive operations."""
    # TODO: Identify sensitive operations (salary, personal data)
    # TODO: Record operation details
    # TODO: Generate alerts for suspicious activity
    pass

def generate_daily_report():
    """TODO: Generate compliance report."""
    # TODO: Summarize access patterns
    # TODO: List sensitive operations
    # TODO: Highlight any anomalies
    pass

## 🌟 Bonus Challenge: Advanced Approval System

Create a complete approval workflow:
- Different approval levels (Manager → Director → C-Level)
- Email notifications for pending approvals
- Time-based auto-rejection (if not approved in 24hrs)
- Approval history and audit trail
- Web dashboard to view and approve requests

**This is open-ended - be creative!**

In [ ]:
# Your code here - this is a complex, production-ready system!
# Consider using:
# - Database for persistent storage
# - Message queue for notifications
# - Scheduled jobs for auto-rejection
# - REST API for approval interface

---
# Conclusion

**Congratulations! You've mastered Agent Hooks in LangChain 1.0!**

You now know how to:
- ✅ Control agent execution with pre_model_hook and post_model_hook
- ✅ Implement authorization and access control
- ✅ Add rate limiting and abuse prevention
- ✅ Track usage and manage costs
- ✅ Create audit trails for compliance
- ✅ Build human-in-the-loop workflows
- ✅ Chain multiple operations for production systems

**Remember:** Hooks are essential for production agents. Always implement proper:
- 🔐 Authorization
- 📋 Audit logging
- ⏱️  Rate limiting
- 💰 Cost tracking
- 🛡️  Security controls

---
**Next Module:** Advanced Agent Patterns and Multi-Agent Systems

**References:**
- [LangChain Documentation](https://python.langchain.com/docs/)
- [LangChain Agents Guide](https://python.langchain.com/docs/concepts/agents/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)